# Mesterséges intelligencia alkalmazási területei ZH - 2025.04.12.

A feladatokat az egyes szöveg cellák után valósítsa meg code cellákban.
Az elkészült Jupyter notebookot nevezze át a saját Neptun kódjára és töltse fel a tárgy Moodle felületén.

A Jupyter notebookot Google Colab környezetben futtassa GPU erőforrások használatával.
A feleadatok megvalósításához a HuggingFace Transformers csomagokat használja.

# 1. Csomag telepítése (5p.)
Telepítse fel a datasets, evaluate, transformers[sentencepiece] csomagokat.


In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00


# 2. Pipeline funkció használata (5p.)
A HF Transformer csomag használatával hozzon létre egy szentiment analízis (sentiment-analysis) funkciót a distilbert/distilbert-base-uncased-finetuned-sst-2-english modell segítségével, és kategorizáljon egy tetszőleges példamondatot.

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis",
                      model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

result = classifier("I love going on a holiday")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

Device set to use cuda:0


label: POSITIVE, with score: 0.9997


# 3. DistilBERT modell finom-hangolása (15p.)

Finomhangolja a DistilBERT modellt az alábbi paraméterek alapján:
*   Modell: distilbert/distilbert-base-uncased
*   Adathalmaz: GLUE gyűjteményből a rte adathalmaz (https://huggingface.co/datasets/nyu-mll/glue#rte)
*   Az adathalmaz betöltésekor használjon dinamikus paddinget
*   A tanítás során használjon validációs adathalmazt és minden epoch végén adja meg a validációs loss és pontosság értékeket
*   A tanítás után értékelje ki a finomhangolt modelt
*   Mentse el a finomhangolt modellt a következő néven: NEPTUN_modell

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
import evaluate
import numpy as np
from transformers import Trainer

#Az adathalmaz beszerzése
raw_datasets = load_dataset("glue", "rte")
checkpoint = "distilbert/distilbert-base-uncased"
print(raw_datasets)

#A tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

#Dinamikus Padding: a data_collator a létrehozott batchekben paddingel
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#A training hyperparaméterei
training_args = TrainingArguments(output_dir="test-trainer",
                                  eval_strategy="epoch",
                                  logging_strategy="epoch",
                                  report_to=[])


#A modell definiálása
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

#Az evaluate definiálása
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "rte")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#A Trainer definiálása, ami elvégzi a tanítást:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    )

#A tanítás elvégzése
trainer.train()

#A tanítás kiértékelése
trainer.evaluate()

#A modell mentése
trainer.save_model("C6TMKF_modell")

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})


Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3456435877.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.696200,0.690046,0.555957
2,0.638000,0.703662,0.595668
3,0.388100,1.024967,0.620939
